In [ ]:
# Copy and paste this code to the Python Interactor in Slicer

import numpy as np

files = [r"j:\Temp\ultraosund_0.npy", r"j:\Temp\segmentation_0.npy", r"j:\Temp\prediction_0.npy"]
volumeNames = ["Ultrasound", "Segmentation", "Prediction"]
labelmapFlags = [False, False, False]
dataTypes = ["Double", "Double", "Float"]

# files = [r"j:\Temp\ultraosund_0.npy", r"j:\Temp\segmentation_0.npy"]
# volumeNames = ["Ultrasound", "Segmentation"]
# labelmapFlags = [False, False]
# dataTypes = ["Double", "Double"]


for fileName, volumeName, labelmap, dataType in zip(files, volumeNames, labelmapFlags, dataTypes):
  print("Importing {} into {}, type {}".format(fileName, volumeName, dataType))
  numpyArray = np.load(fileName)
  importer = vtk.vtkImageImport()
  importer.CopyImportVoidPointer(numpyArray, numpyArray.nbytes)
  setDataType = 'importer.SetDataScalarTypeTo' + dataType + '()'
  eval(setDataType)
  importer.SetNumberOfScalarComponents(1)
  importer.SetWholeExtent(0,numpyArray.shape[0]-1,0,numpyArray.shape[1]-1,0,numpyArray.shape[2]-1)
  importer.SetDataExtentToWholeExtent()
  importer.Update()
  if labelmap:
    volume = slicer.vtkMRMLLabelMapVolumeNode()
  else:
    volume = slicer.vtkMRMLScalarVolumeNode()
  volume.SetName(volumeName)
  # volume.SetLabelMap(labelmap)
  volume.SetAndObserveImageData(importer.GetOutput())
  slicer.mrmlScene.AddNode(volume)
  volumeDisplayNode = 0
  if labelmap == False:
    volumeDisplayNode = slicer.vtkMRMLScalarVolumeDisplayNode()
  else:
    volumeDisplayNode = slicer.vtkMRMLLabelMapVolumeDisplayNode()
  
  slicer.mrmlScene.AddNode(volumeDisplayNode)
  if labelmap == False:
    greyColorTable = slicer.util.getNode('Grey')
    volumeDisplayNode.SetAndObserveColorNodeID(greyColorTable.GetID())
  else:
    genericColorTableNode = slicer.util.getNode('GenericColors')
    volumeDisplayNode.SetAndObserveColorNodeID(genericColorTableNode.GetID())
  
  volume.SetAndObserveDisplayNodeID(volumeDisplayNode.GetID())
  # selectionNode = slicer.app.applicationLogic().GetSelectionNode()
  # selectionNode.SetReferenceActiveVolumeID(volume.GetID())
  # slicer.app.applicationLogic().PropagateVolumeSelection(0)
